In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
dataset = pd.read_csv('Housing Price data set.csv')

In [3]:
dataset.head()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [4]:
dataset.shape

(546, 13)

In [5]:
train_size = math.ceil(546*0.7)
valid_size = 546-train_size
train_size,valid_size

(383, 163)

In [6]:
train_df = dataset[0:384]
train_df.head()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,1,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,2,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,3,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,4,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,5,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [7]:
valid_df = dataset[384:547]
valid_df.head()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
384,385,78000.0,6600,4,2,2,yes,yes,yes,no,no,0,yes
385,386,78900.0,6900,3,1,1,yes,yes,yes,no,no,0,yes
386,387,83900.0,11460,3,1,3,yes,no,no,no,no,2,yes
387,388,85000.0,7020,3,1,1,yes,no,yes,no,yes,2,yes
388,389,85000.0,6540,3,1,1,yes,yes,yes,no,no,2,yes


In [8]:
floor_area_train = train_df['lotsize'].to_numpy()
no_bedroom_train = train_df['bedrooms'].to_numpy()
no_bathroom_train = train_df['bathrms'].to_numpy()
price_train = train_df['price'].to_numpy()

In [9]:
train_dataset=np.stack((np.ones(len(floor_area_train)),floor_area_train,no_bedroom_train,no_bathroom_train),axis=1)
train_dataset.shape

(384, 4)

In [10]:
floor_area_valid = valid_df['lotsize'].to_numpy()
no_bedroom_valid = valid_df['bedrooms'].to_numpy()
no_bathroom_valid = valid_df['bathrms'].to_numpy()
price_valid = valid_df['price'].to_numpy()

In [11]:
valid_dataset=np.stack((np.ones(len(floor_area_valid)),floor_area_valid,no_bedroom_valid,no_bathroom_valid),axis=1)
valid_dataset.shape

(162, 4)

In [13]:
train_dataset = np.transpose(train_dataset)
print(train_dataset.shape)
temp=np.dot(np.transpose(train_dataset),train_dataset)
print(temp.shape,np.linalg.inv(temp).shape)
temp1=np.dot(np.linalg.inv(temp),np.transpose(train_dataset))
coeff = np.dot(temp1,price_train)

(384, 4)
(4, 4) (4, 4)


In [14]:
coeff

array([-5.29966497e+03,  6.10846331e+00,  5.51551435e+03,  1.87576930e+04])

In [15]:
### finding error without regularization
error_without=0
for i in range(162):
    error_without+=abs(coeff[0]+coeff[1]*floor_area_valid[i]+
                    coeff[2]*no_bedroom_valid[i]+coeff[1]*no_bathroom_valid[i]-price_valid[i])/price_valid[i]
error_without/162 *100

38.87782789522317

In [18]:
### finding error with regularization parameter lambda
error_with=100000000009 # just a max_value
matrix_temp = np.dot(np.transpose(train_dataset),price_train)
identity =np.identity(4)
for _lambda in range(50):
    first = np.add(temp,_lambda*identity)
    second = np.linalg.inv(first)
    coeff = np.dot(second,matrix_temp)
    error=0
    for i in range(162):
        error+=abs(coeff[0]+coeff[1]*floor_area_valid[i]+
        coeff[2]*no_bedroom_valid[i]+coeff[1]*no_bathroom_valid[i]-price_valid[i])/price_valid[i]
    error_with=min(error_with,error/162 *100)
error_with

30.65661086883263